# Fallstudie Asset Pricing 2021
## Tobias Schmid, Maximilian Kirsch und Niklas Vogel

### Import der benutzen Bibliotheken

In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
import math

### Import der gegebenen Daten & Übersicht der Datensätze

In [91]:
monthly = pd.read_csv('/Users/NiklasVogel/Downloads/data/monthly_data_1964-2018.csv')
monthly.describe()

,company_id,year,month,day,price,ret,num_shares
count,3.136399e+06,3.136399e+06,3.136399e+06,3.136399e+06,3.098499e+06,3.092872e+06,3.132685e+06
mean,4.430639e+04,1.992430e+03,6.503874e+00,2.996407e+01,3.043882e+01,9.981725e-03,4.489867e+04
std,2.712886e+04,1.373249e+01,3.454350e+00,1.114489e+00,1.336032e+03,1.882013e-01,2.490105e+05
min,1.000000e+00,1.964000e+03,1.000000e+00,2.600000e+01,1.510000e-02,-1.000000e+00,2.000000e+00
25%,1.884800e+04,1.982000e+03,4.000000e+00,2.900000e+01,4.750000e+00,-6.716418e-02,2.920000e+03
50%,4.696100e+04,1.993000e+03,7.000000e+00,3.000000e+01,1.243000e+01,0.000000e+00,8.379000e+03
75%,6.986400e+04,2.003000e+03,1.000000e+01,3.100000e+01,2.489000e+01,7.142857e-02,2.671800e+04
max,8.343700e+04,2.018000e+03,1.200000e+01,3.100000e+01,3.260000e+05,4.700000e+01,2.920640e+07


In [92]:
monthly

,company_id,date,year,month,day,price,ret,num_shares
0,1,31jan1986,1986,1,31,4.375000,NaN,3680.0
1,1,28feb1986,1986,2,28,3.250000,-0.257143,3680.0
2,1,31mar1986,1986,3,31,4.437500,0.365385,3680.0
3,1,30apr1986,1986,4,30,4.000000,-0.098592,3793.0
4,1,30may1986,1986,5,30,3.109375,-0.222656,3793.0
...,...,...,...,...,...,...,...,...
3136394,83437,31aug2018,2018,8,31,301.660004,0.011806,170593.0
3136395,83437,28sep2018,2018,9,28,264.769989,-0.122290,171578.0
3136396,83437,31oct2018,2018,10,31,337.320007,0.274011,171733.0
3136397,83437,30nov2018,2018,11,30,350.480011,0.039013,171733.0


In [93]:
momentum = pd.read_csv('/Users/NiklasVogel/Downloads/data/F-F_Momentum_Factor.CSV')
momentum.describe()

,Unnamed: 0,Mom
count,1131.000000,1131.000000
mean,197369.087533,0.637728
std,2721.967916,4.710608
min,192701.000000,-52.270000
25%,195007.500000,-0.925000
50%,197402.000000,0.780000
75%,199708.500000,2.930000
max,202103.000000,18.360000


In [94]:
momentum

,Unnamed: 0,Mom
0,192701,0.44
1,192702,-1.32
2,192703,3.59
3,192704,4.17
4,192705,2.96
...,...,...
1126,202011,-12.25
1127,202012,-2.42
1128,202101,4.37
1129,202102,-7.68


In [95]:
research = pd.read_csv('/Users/NiklasVogel/Downloads/data/F-F_Research_Data_Factors.CSV')
research.describe()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
count,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000
mean,197343.970976,0.684811,0.207740,0.336086,0.269894
std,2736.602825,5.348188,3.183763,3.509996,0.252052
min,192607.000000,-29.130000,-16.820000,-13.960000,-0.060000
25%,195003.000000,-1.970000,-1.590000,-1.390000,0.030000
50%,197311.000000,1.060000,0.080000,0.140000,0.220000
75%,199707.000000,3.650000,1.770000,1.720000,0.420000
max,202103.000000,38.850000,36.700000,35.460000,1.350000


In [96]:
research

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.30,-2.87,0.22
1,192608,2.64,-1.40,4.19,0.25
2,192609,0.36,-1.32,0.01,0.23
3,192610,-3.24,0.04,0.51,0.32
4,192611,2.53,-0.20,-0.35,0.31
...,...,...,...,...,...
1132,202011,12.47,5.48,2.11,0.01
1133,202012,4.63,4.81,-1.36,0.01
1134,202101,-0.03,7.19,2.85,0.00
1135,202102,2.78,2.11,7.08,0.00


### Aufbereitung & Vereinheitlichung der Datensätze

In [97]:
momentum.rename(columns={'Unnamed: 0': 'date'}, inplace = True)
momentum['date'] = pd.to_datetime(momentum['date'], format = '%Y%m')
momentum.set_index('date', inplace = True)
momentum

,Mom
date,
1927-01-01,0.44
1927-02-01,-1.32
1927-03-01,3.59
1927-04-01,4.17
1927-05-01,2.96
...,...
2020-11-01,-12.25
2020-12-01,-2.42
2021-01-01,4.37


In [98]:
research.rename(columns={'Unnamed: 0': 'date'}, inplace = True)
research['date'] = pd.to_datetime(research['date'], format = '%Y%m')
research.set_index('date', inplace = True)
research

,Mkt-RF,SMB,HML,RF
date,,,,
1926-07-01,2.96,-2.30,-2.87,0.22
1926-08-01,2.64,-1.40,4.19,0.25
1926-09-01,0.36,-1.32,0.01,0.23
1926-10-01,-3.24,0.04,0.51,0.32
1926-11-01,2.53,-0.20,-0.35,0.31
...,...,...,...,...
2020-11-01,12.47,5.48,2.11,0.01
2020-12-01,4.63,4.81,-1.36,0.01
2021-01-01,-0.03,7.19,2.85,0.00


Neues Dataframe 'factors', das alle Faktoren (Mkt-RF, SMB, HML, MOM) und RF beinhaltet

In [99]:
factors = pd.merge(research, momentum, left_index=True, right_index=True)

In [100]:
factors

,Mkt-RF,SMB,HML,RF,Mom
date,,,,,
1927-01-01,-0.06,-0.56,4.83,0.25,0.44
1927-02-01,4.18,-0.10,3.17,0.26,-1.32
1927-03-01,0.13,-1.60,-2.67,0.30,3.59
1927-04-01,0.46,0.43,0.60,0.25,4.17
1927-05-01,5.44,1.41,4.93,0.30,2.96
...,...,...,...,...,...
2020-11-01,12.47,5.48,2.11,0.01,-12.25
2020-12-01,4.63,4.81,-1.36,0.01,-2.42
2021-01-01,-0.03,7.19,2.85,0.00,4.37


### Import Datensatz Fama French Portfolios

Verwendeter Datensatz: "10 Portfolios Formed on Long-Term Reversal" von https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

In [101]:
skiplist = [*range(0,1098)] + [*range(2184,4661)] #Definition der Reihen des Datensatzes, die beim Import nicht benötigt werden
fama_e = pd.read_csv('/Users/NiklasVogel/Downloads/10_Portfolios_Prior_60_13.CSV', skiprows = skiplist)
fama_e

,Unnamed: 0,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR
0,193101,29.68,23.22,19.48,17.59,11.08,10.47,11.86,8.62,9.17,9.42
1,193102,22.36,22.55,20.58,17.19,17.46,8.47,11.04,10.31,12.36,17.78
2,193103,-1.69,-3.43,-9.03,-3.19,-9.46,-9.72,-8.90,-7.61,-7.54,-7.04
3,193104,-21.26,-19.09,-15.53,-12.50,-14.63,-14.59,-15.37,-13.61,-14.14,-12.58
4,193105,-14.91,-9.17,-16.36,-15.42,-13.41,-13.17,-13.21,-14.79,-15.67,-17.05
...,...,...,...,...,...,...,...,...,...,...,...
1079,202012,12.19,10.15,8.63,8.08,6.89,7.43,5.67,5.48,5.89,8.37
1080,202101,36.58,19.56,7.53,4.27,2.76,0.51,0.60,0.20,0.79,2.74
1081,202102,13.14,11.95,9.86,9.70,10.60,9.29,8.03,7.39,7.24,5.05
1082,202103,0.19,2.89,4.31,4.92,5.56,5.51,5.37,4.29,2.54,-0.42


In [102]:
fama_e.rename(columns = {'Unnamed: 0': 'date'}, inplace = True)
fama_e['date'] = pd.to_datetime(fama_e['date'], format = '%Y%m')
fama_e = fama_e.set_index('date')

In [103]:
fama_e

,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR
date,,,,,,,,,,
1931-01-01,29.68,23.22,19.48,17.59,11.08,10.47,11.86,8.62,9.17,9.42
1931-02-01,22.36,22.55,20.58,17.19,17.46,8.47,11.04,10.31,12.36,17.78
1931-03-01,-1.69,-3.43,-9.03,-3.19,-9.46,-9.72,-8.90,-7.61,-7.54,-7.04
1931-04-01,-21.26,-19.09,-15.53,-12.50,-14.63,-14.59,-15.37,-13.61,-14.14,-12.58
1931-05-01,-14.91,-9.17,-16.36,-15.42,-13.41,-13.17,-13.21,-14.79,-15.67,-17.05
...,...,...,...,...,...,...,...,...,...,...
2020-12-01,12.19,10.15,8.63,8.08,6.89,7.43,5.67,5.48,5.89,8.37
2021-01-01,36.58,19.56,7.53,4.27,2.76,0.51,0.60,0.20,0.79,2.74
2021-02-01,13.14,11.95,9.86,9.70,10.60,9.29,8.03,7.39,7.24,5.05


## Regression zur Überprüfung der Strategie

Erstellen eines Dataframes mit allen Daten für den benötigten Zeitraum

In [104]:
test_frame = factors['1931-01-01':'1982-12-01'].join(fama_e['1931-01-01':'1982-12-01'])

In [105]:
test_frame['Lo PRIOR - RF'] = test_frame['Lo PRIOR'] - test_frame['RF']
test_frame['Hi PRIOR - RF'] = test_frame['Hi PRIOR'] - test_frame['RF']

In [106]:
test_frame

,Mkt-RF,SMB,HML,RF,Mom,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR,Lo PRIOR - RF,Hi PRIOR - RF
date,,,,,,,,,,,,,,,,,
1931-01-01,6.24,3.77,7.16,0.15,-10.82,29.68,23.22,19.48,17.59,11.08,10.47,11.86,8.62,9.17,9.42,29.53,9.27
1931-02-01,10.88,3.38,1.61,0.04,-14.37,22.36,22.55,20.58,17.19,17.46,8.47,11.04,10.31,12.36,17.78,22.32,17.74
1931-03-01,-6.43,3.07,-3.66,0.13,9.46,-1.69,-3.43,-9.03,-3.19,-9.46,-9.72,-8.90,-7.61,-7.54,-7.04,-1.82,-7.17
1931-04-01,-9.98,-4.62,-3.94,0.08,11.59,-21.26,-19.09,-15.53,-12.50,-14.63,-14.59,-15.37,-13.61,-14.14,-12.58,-21.34,-12.66
1931-05-01,-13.24,5.17,-6.58,0.09,9.10,-14.91,-9.17,-16.36,-15.42,-13.41,-13.17,-13.21,-14.79,-15.67,-17.05,-15.00,-17.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982-08-01,11.14,-4.09,1.00,0.76,-3.52,5.63,8.14,7.45,7.80,6.50,7.92,6.85,8.77,7.75,8.61,4.87,7.85
1982-09-01,1.29,2.89,0.33,0.51,4.17,4.13,3.39,3.49,3.22,4.29,4.23,4.04,4.51,2.99,1.97,3.62,1.46
1982-10-01,11.30,2.35,-3.73,0.59,0.03,14.25,11.35,11.37,10.39,12.15,12.19,12.04,12.33,14.14,15.70,13.66,15.11


### Regression der Rendite der Flops mit nur einem Faktor (Mkt-RF)

In [107]:
X = test_frame[['Mkt-RF']] 
Y = test_frame['Lo PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Lo PRIOR - RF   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     795.1
Date:                Tue, 29 Jun 2021   Prob (F-statistic):          2.57e-113
Time:                        17:30:58   Log-Likelihood:                -2227.7
No. Observations:                 624   AIC:                             4459.
Df Residuals:                     622   BIC:                             4468.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1086      0.347      3.196      0.001       0.427       1.790
Mkt-RF         1.6603      0.059     28.198      0.000       1.545       1.776
==============================================================================
Omnibus:                      560.054   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19424.039
Skew:                           3.891   Prob(JB):                         0.00
Kurtosis:                      29.201   Cond. No.                         5.93
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression der Rendite der Flops mit Carhart-Factor Model (4 Faktoren)

In [108]:
X = test_frame[['Mkt-RF', 'SMB', 'HML', 'Mom   ']] 
Y = test_frame['Lo PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Lo PRIOR - RF   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     1282.
Date:                Tue, 29 Jun 2021   Prob (F-statistic):          7.75e-298
Time:                        17:30:58   Log-Likelihood:                -1789.3
No. Observations:                 624   AIC:                             3589.
Df Residuals:                     619   BIC:                             3611.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5045      0.179      2.822      0.005       0.153       0.856
Mkt-RF         0.8946      0.035     25.616      0.000       0.826       0.963
SMB            1.8276      0.057     32.129      0.000       1.716       1.939
HML            1.0346      0.054     19.009      0.000       0.928       1.142
Mom           -0.1955      0.042     -4.637      0.000      -0.278      -0.113
==============================================================================
Omnibus:                      348.123   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6690.453
Skew:                           2.049   Prob(JB):                         0.00
Kurtosis:                      18.509   Cond. No.                         7.45
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression der Rendite der Tops mit nur einem Faktor (Mkt-RF)

In [109]:
X = test_frame[['Mkt-RF']] 
Y = test_frame['Hi PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X) 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Hi PRIOR - RF   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.857
Method:                 Least Squares   F-statistic:                     3741.
Date:                Tue, 29 Jun 2021   Prob (F-statistic):          2.71e-265
Time:                        17:30:58   Log-Likelihood:                -1532.6
No. Observations:                 624   AIC:                             3069.
Df Residuals:                     622   BIC:                             3078.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1143      0.114     -1.004      0.316      -0.338       0.109
Mkt-RF         1.1821      0.019     61.164      0.000       1.144       1.220
==============================================================================
Omnibus:                       98.903   Durbin-Watson:                   1.708
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1078.144
Skew:                          -0.266   Prob(JB):                    7.65e-235
Kurtosis:                       9.417   Cond. No.                         5.93
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression der Rendite der Tops mit Carhart-Factor Model (4 Faktoren)

In [110]:
X = test_frame[['Mkt-RF', 'SMB', 'HML', 'Mom   ']] 
Y = test_frame['Hi PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Hi PRIOR - RF   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.897
Method:                 Least Squares   F-statistic:                     1354.
Date:                Tue, 29 Jun 2021   Prob (F-statistic):          2.24e-304
Time:                        17:30:58   Log-Likelihood:                -1429.9
No. Observations:                 624   AIC:                             2870.
Df Residuals:                     619   BIC:                             2892.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0253      0.101     -0.252      0.801      -0.223       0.172
Mkt-RF         1.1368      0.020     57.911      0.000       1.098       1.175
SMB            0.4205      0.032     13.151      0.000       0.358       0.483
HML           -0.3040      0.031     -9.935      0.000      -0.364      -0.244
Mom           -0.1203      0.024     -5.076      0.000      -0.167      -0.074
==============================================================================
Omnibus:                      224.497   Durbin-Watson:                   1.621
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3388.881
Skew:                          -1.168   Prob(JB):                         0.00
Kurtosis:                      14.175   Cond. No.                         7.45
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Selbst implementierte Long-Term Reversal Strategie

Ansatz: Sortieren in "Winner" und "Loser", indem das artihmetische Mittel der monatlichen Renditen für den Zeitraum von vor fünf Jahren bis vor einem Jahr gebildet wird und darauf basierend ein Top 10% und Flop 10% Portfolio gebaut wird. Die Einzelwerte gehen gleichgewichtet in die Portfolios ein.

Die Strategie wird von Anfang 2010 bis Ende 2018 für jeden dazwischen liegenden Monat getestet. Damit sind nur Daten ab Anfang 2005 relevant (60 Monate vor 2010).

In [111]:
monthly #verwendetes Dataframe

,company_id,date,year,month,day,price,ret,num_shares
0,1,31jan1986,1986,1,31,4.375000,NaN,3680.0
1,1,28feb1986,1986,2,28,3.250000,-0.257143,3680.0
2,1,31mar1986,1986,3,31,4.437500,0.365385,3680.0
3,1,30apr1986,1986,4,30,4.000000,-0.098592,3793.0
4,1,30may1986,1986,5,30,3.109375,-0.222656,3793.0
...,...,...,...,...,...,...,...,...
3136394,83437,31aug2018,2018,8,31,301.660004,0.011806,170593.0
3136395,83437,28sep2018,2018,9,28,264.769989,-0.122290,171578.0
3136396,83437,31oct2018,2018,10,31,337.320007,0.274011,171733.0
3136397,83437,30nov2018,2018,11,30,350.480011,0.039013,171733.0


In [112]:
monthly_new = monthly.loc[monthly['year'] >= 2005] # Daten vor 2005 sind nicht relevant -> Performace wird verbessert

In [113]:
monthly_new

,company_id,date,year,month,day,price,ret,num_shares
246,2,31jan2005,2005,1,31,6.620000,-0.040580,2595.0
247,2,28feb2005,2005,2,28,6.321000,-0.045166,2599.0
248,2,31mar2005,2005,3,31,7.110000,0.124822,2625.0
249,2,29apr2005,2005,4,29,6.579000,-0.074684,2625.0
250,2,31may2005,2005,5,31,8.020000,0.219030,2625.0
...,...,...,...,...,...,...,...,...
3136394,83437,31aug2018,2018,8,31,301.660004,0.011806,170593.0
3136395,83437,28sep2018,2018,9,28,264.769989,-0.122290,171578.0
3136396,83437,31oct2018,2018,10,31,337.320007,0.274011,171733.0
3136397,83437,30nov2018,2018,11,30,350.480011,0.039013,171733.0


In [114]:
monthly_new['date'] = pd.to_datetime(monthly_new['date'], format = '%d%b%Y')

<ipython-input-114-882b132856ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_new['date'] = pd.to_datetime(monthly_new['date'], format = '%d%b%Y')


In [119]:
len(monthly_new['company_id'].unique())

7905

In [127]:
master_dict = {}
for year in range(2010, 2019):
    master_dict[year]= {} 
    for month in range(1, 13):
        print(f'{year}, {month}') # only for control purposes
        past_av_ret = [] # list that saves past average returns and the corresponding stock/ company
        for company in monthly_new.loc[(monthly_new['date'] >= datetime.datetime(year, month, 1))]['company_id'].unique(): # only considers stocks/ companies that still exist
            help_df = monthly_new.loc[(monthly_new['company_id'] == company) & (monthly_new['date'] >  datetime.datetime(year-5, month, 1)) & (monthly_new['date'] < datetime.datetime(year-1, month, 1))] # only prior returns are considered that are 5 to 1 year ago for each company that still exists and already existed 1 year ago
            if help_df.empty: # can be the case if company did't exist one year ago
                pass
            else:
                av_ret = help_df['ret'].mean() 
                past_av_ret.append([av_ret, company]) 
        past_av_ret.sort(key=lambda x: x[0]) #sorts lists of average returns and corresponding stock/ company by stock (low first)
        master_dict[year][month] = {}
        master_dict[year][month]['past returns'] = past_av_ret
        master_dict[year][month]['past tops'] = past_av_ret[int(9/10*len(past_av_ret)):] # returns and comapany ids of the top 10% stocks
        master_dict[year][month]['past flops'] = past_av_ret[0:int(1/10*len(past_av_ret))] # returns and comapany ids of the flop 10% stocks

2010, 1
2010, 2
2010, 3
2010, 4
2010, 5
2010, 6
2010, 7
2010, 8
2010, 9
2010, 10
2010, 11
2010, 12
2011, 1
2011, 2
2011, 3
2011, 4
2011, 5
2011, 6
2011, 7
2011, 8
2011, 9
2011, 10
2011, 11
2011, 12
2012, 1
2012, 2
2012, 3
2012, 4
2012, 5
2012, 6
2012, 7
2012, 8
2012, 9
2012, 10
2012, 11
2012, 12
2013, 1
2013, 2
2013, 3
2013, 4
2013, 5
2013, 6
2013, 7
2013, 8
2013, 9
2013, 10
2013, 11
2013, 12
2014, 1
2014, 2
2014, 3
2014, 4
2014, 5
2014, 6
2014, 7
2014, 8
2014, 9
2014, 10
2014, 11
2014, 12
2015, 1
2015, 2
2015, 3
2015, 4
2015, 5
2015, 6
2015, 7
2015, 8
2015, 9
2015, 10
2015, 11
2015, 12
2016, 1
2016, 2
2016, 3
2016, 4
2016, 5
2016, 6
2016, 7
2016, 8
2016, 9
2016, 10
2016, 11
2016, 12
2017, 1
2017, 2
2017, 3
2017, 4
2017, 5
2017, 6
2017, 7
2017, 8
2017, 9
2017, 10
2017, 11
2017, 12
2018, 1
2018, 2
2018, 3
2018, 4
2018, 5
2018, 6
2018, 7
2018, 8
2018, 9
2018, 10
2018, 11
2018, 12


Heutige Renditen der Tops und Flops vor fünf Jahren bis vor einem Jahr

In [128]:
for year in master_dict.keys():
    for month in master_dict[year].keys():
        print(f'{year}, {month}')
        top_returns = []
        for element in master_dict[year][month]['past tops']:
            df = monthly_new.loc[(monthly_new['company_id'] == element[1]) & (monthly_new['year']==year) & (monthly_new['month']==month)]
            if df.empty:
                pass
            else:
                top_returns.append(df['ret'].values[0])
        master_dict[year][month]['tops todays returns'] = top_returns

2010, 1
2010, 2
2010, 3
2010, 4
2010, 5
2010, 6
2010, 7
2010, 8
2010, 9
2010, 10
2010, 11
2010, 12
2011, 1
2011, 2
2011, 3
2011, 4
2011, 5
2011, 6
2011, 7
2011, 8
2011, 9
2011, 10
2011, 11
2011, 12
2012, 1
2012, 2
2012, 3
2012, 4
2012, 5
2012, 6
2012, 7
2012, 8
2012, 9
2012, 10
2012, 11
2012, 12
2013, 1
2013, 2
2013, 3
2013, 4
2013, 5
2013, 6
2013, 7
2013, 8
2013, 9
2013, 10
2013, 11
2013, 12
2014, 1
2014, 2
2014, 3
2014, 4
2014, 5
2014, 6
2014, 7
2014, 8
2014, 9
2014, 10
2014, 11
2014, 12
2015, 1
2015, 2
2015, 3
2015, 4
2015, 5
2015, 6
2015, 7
2015, 8
2015, 9
2015, 10
2015, 11
2015, 12
2016, 1
2016, 2
2016, 3
2016, 4
2016, 5
2016, 6
2016, 7
2016, 8
2016, 9
2016, 10
2016, 11
2016, 12
2017, 1
2017, 2
2017, 3
2017, 4
2017, 5
2017, 6
2017, 7
2017, 8
2017, 9
2017, 10
2017, 11
2017, 12
2018, 1
2018, 2
2018, 3
2018, 4
2018, 5
2018, 6
2018, 7
2018, 8
2018, 9
2018, 10
2018, 11
2018, 12


In [129]:
mean_tops = []
for year in master_dict.keys():
    for month in master_dict[year].keys():
        sum_ = 0
        counter = 0
        for element in master_dict[year][month]['tops todays returns']:
            if math.isnan(element):
                pass
            else:
                sum_ += element
                counter += 1
        mean_tops.append([datetime.datetime(year, month, 1), sum_/counter])

In [130]:
for year in master_dict.keys():
    for month in master_dict[year].keys():
        flop_returns = []
        for element in master_dict[year][month]['past flops']:
            df = monthly_new.loc[(monthly_new['company_id'] == element[1]) & (monthly_new['year']==year) & (monthly_new['month']==month)]
            if df.empty:
                pass
            else:
                flop_returns.append(df['ret'].values[0])
        master_dict[year][month]['flops todays returns'] = flop_returns

In [131]:
mean_flops = []
for year in master_dict.keys():
    for month in master_dict[year].keys():
        sum_ = 0
        counter = 0
        for element in master_dict[year][month]['flops todays returns']:
            if math.isnan(element):
                pass
            else:
                sum_ += element
                counter += 1
        mean_flops.append([datetime.datetime(year, month, 1), sum_/counter])

In [132]:
mean_flopsNtops = []
for i in range(0,len(mean_flops)):
    mean_flopsNtops.append([mean_flops[i][0], mean_flops[i][1], mean_tops[i][1]])

In [133]:
returns_reversal_frame = pd.DataFrame(mean_flopsNtops, columns = ['date', 'return flops', 'return tops'])
returns_reversal_frame = returns_reversal_frame.set_index('date')
returns_reversal_frame

,return flops,return tops
date,,
2010-01-01,0.006990,-0.036742
2010-02-01,0.036205,0.025059
2010-03-01,0.077410,0.078796
2010-04-01,0.068183,0.046184
2010-05-01,-0.085861,-0.095166
...,...,...
2018-08-01,0.020349,0.063343
2018-09-01,-0.021233,-0.012608
2018-10-01,-0.097170,-0.134777


Regression der Strategie mit den "selbstgebauten" Portfolios

In [134]:
test_frame_new = factors['2010-01-01':'2018-12-01'].join(returns_reversal_frame)
test_frame_new['return flops'] = test_frame_new['return flops']*100 #
test_frame_new['return tops'] = test_frame_new['return tops']*100
test_frame_new['return flops - RF'] = test_frame_new['return flops'] - test_frame_new['RF']
test_frame_new['return tops - RF'] = test_frame_new['return tops'] - test_frame_new['RF']
test_frame_new

,Mkt-RF,SMB,HML,RF,Mom,return flops,return tops,return flops - RF,return tops - RF
date,,,,,,,,,
2010-01-01,-3.36,0.37,0.33,0.00,-5.36,0.699008,-3.674173,0.699008,-3.674173
2010-02-01,3.40,1.19,3.19,0.00,3.63,3.620455,2.505925,3.620455,2.505925
2010-03-01,6.31,1.44,2.11,0.01,3.71,7.740985,7.879591,7.730985,7.869591
2010-04-01,2.00,4.86,2.91,0.01,3.21,6.818325,4.618366,6.808325,4.608366
2010-05-01,-7.89,0.14,-2.39,0.01,-0.17,-8.586059,-9.516571,-8.596059,-9.526571
...,...,...,...,...,...,...,...,...,...
2018-08-01,3.44,1.15,-3.92,0.16,5.22,2.034941,6.334341,1.874941,6.174341
2018-09-01,0.06,-2.28,-1.63,0.15,-0.06,-2.123348,-1.260755,-2.273348,-1.410755
2018-10-01,-7.68,-4.78,3.46,0.19,-1.84,-9.717018,-13.477728,-9.907018,-13.667728


In [135]:
test_frame_new = test_frame_new.drop(test_frame_new.index[len(test_frame_new)-1]) # Dec 18 returns don't make sense -> drop last row

In [136]:
test_frame_new

,Mkt-RF,SMB,HML,RF,Mom,return flops,return tops,return flops - RF,return tops - RF
date,,,,,,,,,
2010-01-01,-3.36,0.37,0.33,0.00,-5.36,0.699008,-3.674173,0.699008,-3.674173
2010-02-01,3.40,1.19,3.19,0.00,3.63,3.620455,2.505925,3.620455,2.505925
2010-03-01,6.31,1.44,2.11,0.01,3.71,7.740985,7.879591,7.730985,7.869591
2010-04-01,2.00,4.86,2.91,0.01,3.21,6.818325,4.618366,6.808325,4.608366
2010-05-01,-7.89,0.14,-2.39,0.01,-0.17,-8.586059,-9.516571,-8.596059,-9.526571
...,...,...,...,...,...,...,...,...,...
2018-07-01,3.19,-2.21,0.57,0.16,-1.67,-1.476161,-0.574017,-1.636161,-0.734017
2018-08-01,3.44,1.15,-3.92,0.16,5.22,2.034941,6.334341,1.874941,6.174341
2018-09-01,0.06,-2.28,-1.63,0.15,-0.06,-2.123348,-1.260755,-2.273348,-1.410755


## Flops

### Regression der Rendite der Flops mit nur einem Faktor (Mkt-RF)

In [137]:
X = test_frame_new[['Mkt-RF']] 
Y = test_frame_new['return flops - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      return flops - RF   R-squared:                       0.738
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     295.9
Date:                Tue, 29 Jun 2021   Prob (F-statistic):           2.57e-32
Time:                        18:30:56   Log-Likelihood:                -244.86
No. Observations:                 107   AIC:                             493.7
Df Residuals:                     105   BIC:                             499.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3113      0.243     -1.281      0.203      -0.793       0.171
Mkt-RF         1.1215      0.065     17.201      0.000       0.992       1.251
==============================================================================
Omnibus:                        0.442   Durbin-Watson:                   2.128
Prob(Omnibus):                  0.802   Jarque-Bera (JB):                0.134
Skew:                          -0.042   Prob(JB):                        0.935
Kurtosis:                       3.152   Cond. No.                         3.92
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression der Rendite der Flops mit Carhart-Factor Model (4 Faktoren)

In [138]:
X = test_frame_new[['Mkt-RF', 'SMB', 'HML', 'Mom   ']] 
Y = test_frame_new['return flops - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      return flops - RF   R-squared:                       0.890
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     206.5
Date:                Tue, 29 Jun 2021   Prob (F-statistic):           5.78e-48
Time:                        18:30:56   Log-Likelihood:                -198.40
No. Observations:                 107   AIC:                             406.8
Df Residuals:                     102   BIC:                             420.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0548      0.162     -0.338      0.736      -0.376       0.267
Mkt-RF         0.9266      0.046     20.178      0.000       0.835       1.018
SMB            0.7728      0.072     10.773      0.000       0.631       0.915
HML            0.1950      0.076      2.575      0.011       0.045       0.345
Mom           -0.1594      0.056     -2.869      0.005      -0.270      -0.049
==============================================================================
Omnibus:                        5.843   Durbin-Watson:                   1.636
Prob(Omnibus):                  0.054   Jarque-Bera (JB):                5.485
Skew:                          -0.422   Prob(JB):                       0.0644
Kurtosis:                       3.721   Cond. No.                         4.13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Regression der Tops

### Regression der Rendite der Tops mit nur einem Faktor (Mkt-RF)

In [139]:
X = test_frame_new[['Mkt-RF']] 
Y = test_frame_new['return tops - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X) 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       return tops - RF   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.812
Method:                 Least Squares   F-statistic:                     459.6
Date:                Tue, 29 Jun 2021   Prob (F-statistic):           3.81e-40
Time:                        18:30:56   Log-Likelihood:                -238.74
No. Observations:                 107   AIC:                             481.5
Df Residuals:                     105   BIC:                             486.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5214      0.230     -2.271      0.025      -0.977      -0.066
Mkt-RF         1.3200      0.062     21.438      0.000       1.198       1.442
==============================================================================
Omnibus:                        0.260   Durbin-Watson:                   2.197
Prob(Omnibus):                  0.878   Jarque-Bera (JB):                0.376
Skew:                           0.108   Prob(JB):                        0.829
Kurtosis:                       2.805   Cond. No.                         3.92
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression der Rendite der Tops mit Carhart-Factor Model (4 Faktoren)

In [140]:
X = test_frame_new[['Mkt-RF', 'SMB', 'HML', 'Mom   ']] 
Y = test_frame_new['return tops - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X) 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       return tops - RF   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     375.1
Date:                Tue, 29 Jun 2021   Prob (F-statistic):           4.85e-60
Time:                        18:30:56   Log-Likelihood:                -181.38
No. Observations:                 107   AIC:                             372.8
Df Residuals:                     102   BIC:                             386.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2952      0.138     -2.135      0.035      -0.569      -0.021
Mkt-RF         1.1339      0.039     28.951      0.000       1.056       1.212
SMB            0.8192      0.061     13.388      0.000       0.698       0.941
HML           -0.1449      0.065     -2.244      0.027      -0.273      -0.017
Mom           -0.2287      0.047     -4.827      0.000      -0.323      -0.135
==============================================================================
Omnibus:                        1.870   Durbin-Watson:                   1.764
Prob(Omnibus):                  0.393   Jarque-Bera (JB):                1.370
Skew:                           0.121   Prob(JB):                        0.504
Kurtosis:                       3.499   Cond. No.                         4.13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Anhang
Zur Überprüfung der eigenen Ergebnissen werden die Regressionen für den Zeitraum von 2010-2018 nochmals mit den "fertigen" Portfolios der Long Reversal Strategie von Fama French auf 

In [141]:
comparison_frame = factors['2010-01-01':'2018-12-01'].join(fama_e['2010-01-01':'2018-12-01'])

In [142]:
comparison_frame

,Mkt-RF,SMB,HML,RF,Mom,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR
date,,,,,,,,,,,,,,,
2010-01-01,-3.36,0.37,0.33,0.00,-5.36,4.72,0.66,-0.91,-0.70,-2.34,-2.33,-2.31,-2.99,-2.61,-4.37
2010-02-01,3.40,1.19,3.19,0.00,3.63,3.59,5.85,3.97,4.35,3.75,4.58,3.47,3.41,2.65,2.88
2010-03-01,6.31,1.44,2.11,0.01,3.71,14.64,8.68,8.19,7.55,8.03,6.99,6.18,6.03,5.50,7.69
2010-04-01,2.00,4.86,2.91,0.01,3.21,16.12,12.51,8.22,6.78,5.93,4.99,4.43,3.77,3.74,3.70
2010-05-01,-7.89,0.14,-2.39,0.01,-0.17,-12.23,-8.82,-6.99,-7.02,-7.20,-6.63,-6.78,-7.33,-7.08,-7.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-01,3.44,1.15,-3.92,0.16,5.22,1.40,1.71,4.32,3.68,3.15,1.99,2.82,2.23,3.89,4.45
2018-09-01,0.06,-2.28,-1.63,0.15,-0.06,-1.38,-1.44,-1.82,-1.64,-2.05,-1.45,-2.15,-1.88,-1.74,-2.06
2018-10-01,-7.68,-4.78,3.46,0.19,-1.84,-13.08,-9.27,-8.70,-7.42,-9.04,-7.71,-8.89,-7.98,-8.17,-9.94


In [143]:
comparison_frame['Lo PRIOR - RF'] = comparison_frame['Lo PRIOR'] - comparison_frame['RF']
comparison_frame

,Mkt-RF,SMB,HML,RF,Mom,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR,Lo PRIOR - RF
date,,,,,,,,,,,,,,,,
2010-01-01,-3.36,0.37,0.33,0.00,-5.36,4.72,0.66,-0.91,-0.70,-2.34,-2.33,-2.31,-2.99,-2.61,-4.37,4.72
2010-02-01,3.40,1.19,3.19,0.00,3.63,3.59,5.85,3.97,4.35,3.75,4.58,3.47,3.41,2.65,2.88,3.59
2010-03-01,6.31,1.44,2.11,0.01,3.71,14.64,8.68,8.19,7.55,8.03,6.99,6.18,6.03,5.50,7.69,14.63
2010-04-01,2.00,4.86,2.91,0.01,3.21,16.12,12.51,8.22,6.78,5.93,4.99,4.43,3.77,3.74,3.70,16.11
2010-05-01,-7.89,0.14,-2.39,0.01,-0.17,-12.23,-8.82,-6.99,-7.02,-7.20,-6.63,-6.78,-7.33,-7.08,-7.19,-12.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-01,3.44,1.15,-3.92,0.16,5.22,1.40,1.71,4.32,3.68,3.15,1.99,2.82,2.23,3.89,4.45,1.24
2018-09-01,0.06,-2.28,-1.63,0.15,-0.06,-1.38,-1.44,-1.82,-1.64,-2.05,-1.45,-2.15,-1.88,-1.74,-2.06,-1.53
2018-10-01,-7.68,-4.78,3.46,0.19,-1.84,-13.08,-9.27,-8.70,-7.42,-9.04,-7.71,-8.89,-7.98,-8.17,-9.94,-13.27


In [144]:
X = comparison_frame[['Mkt-RF']] 
Y = comparison_frame['Lo PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Lo PRIOR - RF   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.625
Method:                 Least Squares   F-statistic:                     179.4
Date:                Tue, 29 Jun 2021   Prob (F-statistic):           1.56e-24
Time:                        18:30:57   Log-Likelihood:                -306.37
No. Observations:                 108   AIC:                             616.7
Df Residuals:                     106   BIC:                             622.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9925      0.415     -2.393      0.018      -1.815      -0.170
Mkt-RF         1.4525      0.108     13.392      0.000       1.237       1.668
==============================================================================
Omnibus:                        5.403   Durbin-Watson:                   1.718
Prob(Omnibus):                  0.067   Jarque-Bera (JB):                4.940
Skew:                           0.407   Prob(JB):                       0.0846
Kurtosis:                       3.659   Cond. No.                         3.97
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [145]:
X = comparison_frame[['Mkt-RF', 'SMB', 'HML', 'Mom   ']] 
Y = comparison_frame['Lo PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Lo PRIOR - RF   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     89.09
Date:                Tue, 29 Jun 2021   Prob (F-statistic):           1.49e-32
Time:                        18:30:57   Log-Likelihood:                -279.11
No. Observations:                 108   AIC:                             568.2
Df Residuals:                     103   BIC:                             581.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5770      0.332     -1.738      0.085      -1.235       0.081
Mkt-RF         1.1825      0.092     12.817      0.000       1.000       1.366
SMB            0.9400      0.149      6.315      0.000       0.645       1.235
HML            0.5520      0.157      3.516      0.001       0.241       0.863
Mom           -0.2521      0.115     -2.187      0.031      -0.481      -0.023
==============================================================================
Omnibus:                        3.326   Durbin-Watson:                   1.572
Prob(Omnibus):                  0.190   Jarque-Bera (JB):                3.297
Skew:                           0.146   Prob(JB):                        0.192
Kurtosis:                       3.805   Cond. No.                         4.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [146]:
comparison_frame['Hi PRIOR - RF'] = comparison_frame['Hi PRIOR'] - comparison_frame['RF']

In [147]:
X = comparison_frame[['Mkt-RF']] 
Y = comparison_frame['Hi PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Hi PRIOR - RF   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     617.5
Date:                Tue, 29 Jun 2021   Prob (F-statistic):           5.18e-46
Time:                        18:30:57   Log-Likelihood:                -214.99
No. Observations:                 108   AIC:                             434.0
Df Residuals:                     106   BIC:                             439.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1439      0.178     -0.809      0.420      -0.497       0.209
Mkt-RF         1.1564      0.047     24.849      0.000       1.064       1.249
==============================================================================
Omnibus:                        1.869   Durbin-Watson:                   2.187
Prob(Omnibus):                  0.393   Jarque-Bera (JB):                1.546
Skew:                           0.134   Prob(JB):                        0.462
Kurtosis:                       2.479   Cond. No.                         3.97
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [148]:
X = comparison_frame[['Mkt-RF', 'SMB', 'HML', 'Mom   ']] 
Y = comparison_frame['Hi PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Hi PRIOR - RF   R-squared:                       0.961
Model:                            OLS   Adj. R-squared:                  0.959
Method:                 Least Squares   F-statistic:                     630.0
Date:                Tue, 29 Jun 2021   Prob (F-statistic):           1.98e-71
Time:                        18:30:57   Log-Likelihood:                -143.89
No. Observations:                 108   AIC:                             297.8
Df Residuals:                     103   BIC:                             311.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0220      0.095      0.232      0.817      -0.166       0.210
Mkt-RF         0.9988      0.026     37.862      0.000       0.946       1.051
SMB            0.7070      0.043     16.613      0.000       0.623       0.791
HML           -0.0890      0.045     -1.982      0.050      -0.178    5.93e-05
Mom           -0.1064      0.033     -3.229      0.002      -0.172      -0.041
==============================================================================
Omnibus:                        2.141   Durbin-Watson:                   1.826
Prob(Omnibus):                  0.343   Jarque-Bera (JB):                1.546
Skew:                           0.059   Prob(JB):                        0.462
Kurtosis:                       2.426   Cond. No.                         4.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""